In [1]:
import numpy as np
from matplotlib import cm
from matplotlib import colors
import matplotlib.pyplot as plt
from halotools.sim_manager import CachedHaloCatalog
from halotools.mock_observables import wp
from halotools.empirical_models import PrebuiltHodModelFactory
from tabcorr import TabCorr

import pandas as pd
from halotools.sim_manager import UserSuppliedHaloCatalog, UserSuppliedPtclCatalog,FakeSim
from halotools.mock_observables import delta_sigma, wp, return_xyz_formatted_array
from halotools.empirical_models import NFWProfile
from halotools.empirical_models import PrebuiltHodModelFactory, HodModelFactory
from halotools.empirical_models import AssembiasZheng07Cens, AssembiasZheng07Sats, TrivialPhaseSpace, NFWPhaseSpace
from halotools.utils import add_halo_hostid

import time

In [2]:
def create_halo_and_particle_catalogs_for_halotools(halo_catalog_path, particle_catalog_path):
    halo_df = pd.read_csv(halo_catalog_path)
    particles_df = pd.read_csv(particle_catalog_path, delimiter =' +',  names=['x','y','z'], engine='python')
    print('Files read.')
    
    ptcl_x = particles_df['x'].values
    ptcl_y = particles_df['y'].values
    ptcl_z = particles_df['z'].values

    particle_mass = 1.51e9
    num_ptcl_per_dim = 3840
    x = halo_df['x'].values
    y = halo_df['y'].values
    z = halo_df['z'].values
    vx = halo_df['vx'].values
    vy = halo_df['vy'].values
    vz = halo_df['vz'].values
    mass = halo_df['Mvir'].values
    radius = halo_df['Rvir'].values/1e3 #convert to Mpc
    ids = np.arange(0, len(halo_df))
    upid = halo_df['upId'].values
    simname = 'MDPL2'
    
    #get concentrations
#     nfw = NFWProfile(redshift=redshift, cosmology = Planck15, mdef = 'vir', conc_mass_model = 'dutton_maccio14')
#     model_conc = nfw.conc_NFWmodel(prim_haloprop = mass)
    concentrations = halo_df['Rvir'].values / halo_df['Rs'].values
    
    print('Creating catalogs...')
    particle_catalog = UserSuppliedPtclCatalog(x = ptcl_x, y = ptcl_y, z = ptcl_z, Lbox = Lbox, particle_mass = particle_mass,
                                  redshift = redshift)
    halo_catalog = UserSuppliedHaloCatalog(user_supplied_ptclcat = particle_catalog, redshift = redshift, simname = simname,
                                       Lbox = Lbox, particle_mass = particle_mass, num_ptcl_per_dim =num_ptcl_per_dim,
                                       halo_x = x, halo_y = y, halo_z = z,
                                       halo_vx = vx, halo_vy = vy, halo_vz = vz, 
                                       halo_id = ids, halo_mvir = mass, halo_rvir = radius,
                                       halo_nfw_conc = concentrations, halo_upid = upid )
    
    #add hostid
    add_halo_hostid(halo_catalog.halo_table)
    
    return halo_catalog, particle_catalog

In [3]:
data_directory = '/Users/fardila/Documents/Data/baryonic_effects/CMASS/'
queried_halo_cat_file = 'halo_catalogs/mdpl2_hlist_0.65650_Mvir11.2.csv'
test_halo_cat_file = 'halo_catalogs/test.csv'
### "row_id","Mvir","Rvir","M200c","M500c","x","y","z","scale"
# full_halo_cat_file = 'halo_catalogs/cut_halo_df.pkl'
particle_cat_file = 'particle_catalogs/mdpl2_particles_0.6565_10m.dat'
test_particle_cat_file = 'particle_catalogs/test.dat'
### "x","y","z"
displacedA_particle_cat_file = 'particle_catalogs/MDPL2_bfc_particles_A.out'
displacedB_particle_cat_file = 'particle_catalogs/MDPL2_bfc_particles_B.out'
displacedC_particle_cat_file = 'particle_catalogs/MDPL2_bfc_particles_C.out'

In [4]:
redshift = (1./0.65650)-1. #z=0.523
Lbox = 1000. #Mpc/h

In [ ]:
halo_catalog, particle_catalog = create_halo_and_particle_catalogs_for_halotools(data_directory+queried_halo_cat_file,
                                                                                 data_directory+particle_cat_file)

Files read.
Creating catalogs...


In [ ]:
#add to cache
halo_catalog.add_halocat_to_cache(data_directory+'halo_catalogs/mdpl2_hlist_0.65650_Mvir11.2.hdf5', simname='mdpl2',
                                 halo_finder='rockstar', version_name = 'Mvir_gtr_11.2', 
                                  processing_notes=' ',scale=0.65650)

# precompute 

In [ ]:
time1 = time.time()

# First, we tabulate the correlation functions in the halo catalog.
rp_bins = np.logspace(-1, 1, 20)

halotab = TabCorr.tabulate(halo_catalog, wp, rp_bins, pi_max=80,
                           period=halo_catalog.Lbox)

# We can save the result for later use.
halotab.write('mdpl2_tabCorr.hdf5')

print('{0} seconds'.format(time.time() - time1)) 

In [ ]:
# We could read it in like this. Thus, we can skip the previous steps in the
# future.
halotab = TabCorr.read('mdpl2_tabCorr.hdf5')

# Now, we're ready to calculate correlation functions for a specific model.
model = PrebuiltHodModelFactory('zheng07', threshold=-18)